In [ ]:
require_once '/vendor/autoload.php';

### 利用するディレクトリやファイルパスの設定

In [ ]:
$DATA_DIR = './data/';
$MODEL_FILE = './model/mnist-svm.phpml';

### ModelManager を作成する
学習したモデルをファイルに保存したり、学習済みモデルのファイルを読み込んだりするためのクラス

In [ ]:
use Phpml\ModelManager;

$modelManager = new ModelManager();

### データセットを読み込む
- trainDataset: モデルの学習に使用するためのデータセット
- testDataset: 学習したモデルの評価に使うためのデータセット

In [ ]:
use Phpml\Dataset\MnistDataset;

$trainDataset = new MnistDataset($DATA_DIR . 'train-images-idx3-ubyte', $DATA_DIR . 'train-labels-idx1-ubyte');
$testDataset = new MnistDataset($DATA_DIR . 't10k-images-idx3-ubyte', $DATA_DIR . 't10k-labels-idx1-ubyte');

### モデルを作成する
今回は分類アルゴリズムとしてサポートベクターマシン (SVM) を利用する。

PHP-ML では、SVM の分類器は `SVC` と言う名前で提供されている。

In [ ]:
use Phpml\Classification\SVC;

$classifier = new SVC();

### モデルを学習する
1時間くらいかかった

In [ ]:
$start = microtime(true);
$classifier->train($trainDataset->getSamples(), $trainDataset->getTargets());
echo sprintf('Time: %.1f sec', microtime(true) - $start);

### (学習したモデルをファイルに保存する) 

In [ ]:
$modelManager->saveToFile($classifier, $MODEL_FILE);

### (保存した学習済みモデルを読み込む)

In [ ]:
$classifier = $modelManager->restoreFromFile($MODEL_FILE);

### テストデータに対して予測を行う

In [ ]:
$start = microtime(true);
$predicted = $classifier->predict($testDataset->getSamples());
echo sprintf('Time: %.1f sec', microtime(true) - $start);

### 性能を評価する

In [ ]:
use Phpml\Metric\Accuracy;

echo sprintf('Accuracy: %.2f %%', Accuracy::score($testDataset->getTargets(), $predicted) * 100);

### 不正解パターンを確認する

In [ ]:
$actual = $testDataset->getTargets();

$results = [];
for ($i=0; $i < count($predicted); $i++) {
    if ($predicted[$i] != $actual[$i]) {
        $results[] = sprintf('(%04d) Predict: %s, Actual: %s', $i, $predicted[$i], $actual[$i]);
    }
}
echo join("\n", $results);

### メモリ使用量を確認する

In [ ]:
$size = memory_get_peak_usage(true);
$unit=['B','KB','MB','GB','TB','PB'];
echo sprintf('Memory: %s', @round($size/pow(1024, ($i=floor(log($size, 1024)))), 2).' '.$unit[$i]);

### 画像を無理やり表示する

In [ ]:
function displayMNIST($arr) {
    assert(count($arr) == 784);
    $i = 0;
    $str = "";
    foreach ($arr as $item) {
        assert($item >= 0 and $item <= 1);
        $str .= $item < 0.5 ? '　' : '■';
        if (++$i % 28 == 0) $str .= "\n";
    }
    return $str;
}

In [ ]:
echo displayMNIST($testDataset->getSamples()[1]);